In [1]:
#load packages
import pandas as pd
import json

In [6]:
#load data
path = r"/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Labels/coco/zooupdate1_coco.json"
with open(path) as f:
  cocozoo = json.load(f)

dgc = []
for ann in cocozoo:
    if ann['category'] == 'Other Bird' or ann['category'] == 'Seagull':
        continue
    else:
        dgc.append(ann)
dgc

[{'annotation_id': 1,
  'bbox': [441.8125, 333.4375, 91, 80],
  'area': 7280,
  'category_id': 1,
  'category': 'Duck',
  'image_id': 1,
  'filename': 'BDA_18A4_20181107_3_01_06.png',
  'labeler_id': 1},
 {'annotation_id': 2,
  'bbox': [128.79855346679688,
   94.8018569946289,
   95.471923828125,
   126.29488372802734],
  'area': 12057.61551916413,
  'category_id': 1,
  'category': 'Duck',
  'image_id': 2,
  'filename': 'BDA_18A4_20181106_3_07_06.png',
  'labeler_id': 2},
 {'annotation_id': 3,
  'bbox': [369.1244201660156,
   126.650146484375,
   83.4007568359375,
   173.5181884765625],
  'area': 14471.548243746161,
  'category_id': 1,
  'category': 'Duck',
  'image_id': 2,
  'filename': 'BDA_18A4_20181106_3_07_06.png',
  'labeler_id': 2},
 {'annotation_id': 4,
  'bbox': [468.98583984375,
   142.0250701904297,
   104.25091552734375,
   172.4199981689453],
  'area': 17974.942664335482,
  'category_id': 1,
  'category': 'Duck',
  'image_id': 2,
  'filename': 'BDA_18A4_20181106_3_07_06.pn